In [1]:
import numpy as np
import pandas as pd

import meet_indicators
import default_parameters
import utils
import ca_reopening_tiers

county_state_name = default_parameters.county_state_name
fulldate_format = default_parameters.fulldate_format
time_zone = default_parameters.time_zone
start_date = default_parameters.start_date
yesterday_date = default_parameters.yesterday_date
today_date = default_parameters.today_date
one_week_ago = default_parameters.one_week_ago
two_weeks_ago = default_parameters.two_weeks_ago
three_weeks_ago = default_parameters.three_weeks_ago
two_days_ago = default_parameters.two_days_ago
three_days_ago = default_parameters.three_days_ago
eight_days_ago = default_parameters.eight_days_ago
nine_days_ago = default_parameters.nine_days_ago

In [2]:
a1 = ca_reopening_tiers.case_rate(county_state_name, start_date, "today")
a2 = ca_reopening_tiers.case_rate(county_state_name, start_date, "one_week_ago")
a3 = ca_reopening_tiers.case_rate(county_state_name, start_date, "two_weeks_ago")

In [3]:
print(a1)
print(a2)
print(a3)

new_cases    8.13
dtype: float64
new_cases    9.82
dtype: float64
new_cases    12.36
dtype: float64


In [4]:
b1 = ca_reopening_tiers.positive_rate(start_date, "one_week_ago")
b2 = ca_reopening_tiers.positive_rate(start_date, "two_weeks_ago")

In [5]:
print(b1)
print(b2)

0.034
0.039


In [6]:
c1 = ca_reopening_tiers.test_rate(start_date, "one_week_ago")
c2 = ca_reopening_tiers.test_rate(start_date, "two_weeks_ago")

In [7]:
print(c1)
print(c2)

County_Performed    234.61
dtype: float64
County_Performed    269.35
dtype: float64
